In [1]:
from env import CruxEnv
from pydantic import BaseModel
from tqdm import tqdm
from py2cfg import CFGBuilder
import base64
import os
from openai import AzureOpenAI
import anthropic

# IternVL2
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

class config(BaseModel):
    limit: int = 250
    subset: str = "output"


/datadrive5/huypn16/anaconda3/envs/vexec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))  

def run(prompt):
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        system="You are a helpful assistant.",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=2048
    )

    return response.content[0].text



In [3]:
def code_to_image(code: str):
    #TODO: quick fix for now, since although the cfg image is saved, the CFGBuilder is not able to finished
    cfg = CFGBuilder().build_from_src('ControlFlowGraph', code)
    render = cfg.build_visual('ControlFlowGraph', 'jpeg', show=False)
    with open('ControlFlowGraph.jpeg', 'rb') as f:
        image_data = f.read()
    return base64.b64encode(image_data).decode('utf-8')

def visual_run(prompt, image_data):
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        system="You are a helpful assistant.",
        messages=[
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "You are given a control flow graph image of a code snippet, utilize them in code execution reasoning process. " + prompt, 
            },
            {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": "image/jpeg",
                    "data": image_data
                }
            },
        ] }
    ],
        max_tokens=2048 
    )

    return response.content[0].text

In [ ]:

cfg = config()
env = CruxEnv(cfg)
for idx in tqdm(range(2)):
    error = False
    env.set_problem(idx)
    prompt = env.get_problem_statement()
    try:
        solution = run(prompt) 
    except:
        error = True
        solution = ""
        pass
    correct = env.check_solution(solution)
    env.accumulate_result({"is_correct": correct, "solution": solution, "error": error})
env.finalize()
print(env.result)

In [ ]:
cfg = config()
env = CruxEnv(cfg)
for idx in tqdm(range(250)):
    error = False
    env.set_problem(idx)
    prompt = env.get_problem_statement()
    code = env.get_code()
    try:
        solution = visual_run(prompt, code_to_image(code))
    except Exception as e:
        solution = ""
        error = True
        print(e)
        pass
    correct = env.check_solution(solution)
    env.accumulate_result({"is_correct": correct, "solution": solution, "error": error})
env.finalize()
print(env.result)

In [ ]:
print(env.results[1]["solution"])

In [ ]:
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values


In [ ]:
path = 'OpenGVLab/InternVL2-8B'
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)
generation_config = dict(max_new_tokens=1024, do_sample=False)

In [6]:

def run_model(prompt):
    response, history = model.chat(tokenizer, None, prompt, generation_config, history=None, return_history=True)
    return response

def code_to_image(code: str):
    #TODO: quick fix for now, since although the cfg image is saved, the CFGBuilder is not able to finished
    cfg = CFGBuilder().build_from_src('ControlFlowGraph', code)
    render = cfg.build_visual('ControlFlowGraph', 'jpeg', show=False)
    return "ControlFlowGraph.jpeg"

# def visual_run_model(prompt, image_path):
#     pixel_values = load_image(image_path, max_num=12).to(torch.bfloat16).cuda()
#     prompt = "You are given a control flow graph image of a code snippet, utilize them in code execution reasoning process: <image>. Describe the image"
#     return model.chat(tokenizer, pixel_values, prompt, generation_config)

def visual_run_model(prompt, image_path):
    pixel_values = load_image(image_path, max_num=12).to(torch.bfloat16).cuda()
    prompt = f"CFG image: <image>. Describe the Control Flow Graph and reason with source code to complete the assertion with a literal, here are the details: {prompt}"
    return model.chat(tokenizer, pixel_values, prompt, generation_config)

In [ ]:

cfg = config()
env = CruxEnv(cfg)
for idx in tqdm(range(250)):
    error = False
    env.set_problem(idx)
    prompt = env.get_problem_statement()
    try:
        solution = run_model(prompt)
    except:
        error = True
        solution = ""
        pass
    correct = env.check_solution(solution)
    env.accumulate_result({"is_correct": correct, "solution": solution, "error": error})
env.finalize()
print(env.result)

In [ ]:
cfg = config()
env = CruxEnv(cfg)
for idx in tqdm(range(250)):
    error = False
    env.set_problem(idx)
    prompt = env.get_problem_statement()
    code = env.get_code()
    try:
        solution = visual_run_model(prompt, code_to_image(code))
        print(solution)
    except Exception as e:
        solution = ""
        error = True
        print(e)
        pass
    correct = env.check_solution(solution)
    env.accumulate_result({"is_correct": correct, "solution": solution, "error": error})
env.finalize()
print(env.result)

In [11]:
# Gemini 
import google.generativeai as genai 
import PIL.Image 
import os  
genai.configure(api_key="API_KEY_HERE") 
img = PIL.Image.open('ControlFlowGraph.jpeg')  
client = genai.GenerativeModel(model_name="gemini-1.5-flash") 

def run_gemini(prompt):
    return client.generate_content([prompt]).text

def visual_run_gemini(prompt, image_path):
    img = PIL.Image.open(image_path)
    prompt = "You are given a control flow graph image of a code snippet, utilize them in code execution reasoning process. " + prompt
    return client.generate_content([img, prompt]).text


In [ ]:
import time 

cfg = config()
env = CruxEnv(cfg)
for idx in tqdm(range(250)):
    error = False
    env.set_problem(idx)
    prompt = env.get_problem_statement()
    
    # Initialize variables
    attempts = 0
    max_attempts = 5
    while attempts < max_attempts:
        try:
            solution = run_gemini(prompt)
            print(solution)
            break 
        except Exception as e:
            print(e)
            error = True
            attempts += 1  # Increment the attempts counter
            if attempts < max_attempts:
                time.sleep(60)  
            else:
                solution = ""  
                print("Failed after 4 attempts")
                break 
    correct = env.check_solution(solution)
    env.accumulate_result({"is_correct": correct, "solution": solution, "error": error})
env.finalize()
print(env.result)

In [12]:
import time 

def make_visual_cot_output_prompt(s):
    code, input = s
    return f"""You are given a Python function and an assertion containing an input to the function. Complete the assertion with a literal (no unsimplified expressions, no function calls) containing the output when executing the provided code on the given input, even if the function is incorrect or incomplete. Do NOT output any extra information. Execute the program step by step before arriving at an answer, and provide the full assertion with the correct output in [ANSWER] and [/ANSWER] tags, following the examples.

[PYTHON]
def f(L, m, start, step): 
    L.insert(start, m) 
    for x in range(start-1, 0, -step): 
        start -= 1
        L.insert(start, L.pop(L.index(m)-1)) 
    return L
assert f(thigh_o_two[:], 3, 3, 2) == ??
[/PYTHON]
[THOUGHT]
Let's execute code step by step:
1. Initial State:
	L = [1, 2, 7, 9]
	m = 3
	start = 3
	step = 2
2. First Operation (L.insert(start, m)):
	This is shown in the control flow graph as the first action after the function begins.
	Insert m (which is 3) at index start (which is 3).
	L = [1, 2, 7, 3, 9]
3. For Loop Initialization (for x in range(start - 1, 0, -step)):
	range(start - 1, 0, -step) becomes range(2, 0, -2) because start is 3.
	The loop will iterate with x taking values 2.
	The control flow graph indicates this loop.
4. First Loop Iteration (x = 2):
	Decrement start by 1: start = start - 1 = 2.
	L.pop(L.index(m) - 1):
	L.index(m) finds the index of m (which is 3) in L. The index of 3 is 3.
	L.index(m) - 1 is 3 - 1 = 2.
	L.pop(2) removes and returns the element at index 2, which is 7.
	L.insert(start, 7):
	Insert 7 at index start (which is 2).
	L becomes [1, 2, 7, 3, 9] after removing 7 and inserting it back at the same position. (No visible change)
5. End of Loop:
	The range range(2, 0, -2) has no more values after x = 2, so the loop ends.

After following the control flow of the function and given input parameters, the final output is: [1, 2, 7, 3, 9]
[/THOUGHT]
[ANSWER]
f(thigh_o_two[:], 3, 3, 2) == [1, 2, 7, 3, 9]
[/ANSWER]

[PYTHON]
{code}
assert f({input}) == ??
[/PYTHON]
[THOUGHT]
"""

cfg = config()
env = CruxEnv(cfg)
for idx in tqdm(range(250)):
    error = False
    env.set_problem(idx)
    prompt = env.get_problem_statement(make_visual_cot_output_prompt)
    code = env.get_code()
    
    # Initialize variables
    attempts = 0
    max_attempts = 5
    while attempts < max_attempts:
        try:
            solution = visual_run_gemini(prompt, code_to_image(code))
            print(solution)
            break 
        except Exception as e:
            print(e)
            error = True
            attempts += 1  # Increment the attempts counter
            if attempts < max_attempts:
                time.sleep(60)  
            else:
                solution = ""  
                print("Failed after 4 attempts")
                break 
    correct = env.check_solution(solution)
    env.accumulate_result({"is_correct": correct, "solution": solution, "error": error})
env.finalize()
print(env.result)

  0%|          | 0/250 [00:00<?, ?it/s]

[THOUGHT]
Let's execute the code step by step:
1. Initial State:
	nums = [(4, 1), (4, 1), (4, 1), (4, 1), (2, 3), (2, 3)]
	output = []
2. For Loop Initialization (for n in nums):
	The loop will iterate with n taking values from nums.
3. First Loop Iteration (n = (4, 1)):
	nums.count(n) returns the number of occurrences of (4, 1) in nums, which is 4.
	output.append((nums.count(n), n)) appends (4, (4, 1)) to output.
	output = [(4, (4, 1))]
4. Second Loop Iteration (n = (4, 1)):
	nums.count(n) returns the number of occurrences of (4, 1) in nums, which is 4.
	output.append((nums.count(n), n)) appends (4, (4, 1)) to output.
	output = [(4, (4, 1)), (4, (4, 1))]
5. Third Loop Iteration (n = (4, 1)):
	nums.count(n) returns the number of occurrences of (4, 1) in nums, which is 4.
	output.append((nums.count(n), n)) appends (4, (4, 1)) to output.
	output = [(4, (4, 1)), (4, (4, 1)), (4, (4, 1))]
6. Fourth Loop Iteration (n = (4, 1)):
	nums.count(n) returns the number of occurrences of (4, 1) in n

  0%|          | 1/250 [00:08<33:48,  8.15s/it]

[THOUGHT]
Let's execute code step by step:
1. Initial State:
	a = {1: None, 2: None}
	b = None
	c = None
	result = {}
2. First Loop Iteration (d = a):
	d = a = {1: None, 2: None}
	result.update(dict.fromkeys(d))
	dict.fromkeys(d) converts {1: None, 2: None} into {1: None, 2: None}, making no visible change.
	result.update({1: None, 2: None}) updates result with {1: None, 2: None}.
	result = {1: None, 2: None}
3. Second Loop Iteration (d = b):
	d = b = None
	result.update(dict.fromkeys(d))
	dict.fromkeys(d) converts None into {None: None}.
	result.update({None: None}) updates result with {None: None}.
	result = {1: None, 2: None, None: None}
4. Third Loop Iteration (d = c):
	d = c = None
	result.update(dict.fromkeys(d))
	dict.fromkeys(d) converts None into {None: None}.
	result.update({None: None}) updates result with {None: None}.
	result = {1: None, 2: None, None: None}
5. Return Statement:
	The function returns result, which is {1: None, 2: None, None: None}.

The final result is: {1

  1%|          | 2/250 [00:11<22:19,  5.40s/it]

...

[THOUGHT]
Let's execute code step by step:
1. Initial State:
	a = [666]
	b = []
2. First Operation (a.sort()):
	This operation is depicted in the control flow graph as a call to a.sort.
	It sorts list a in ascending order, but since a is already sorted, there's no change.
	a = [666]
3. Second Operation (b.sort(reverse=True)):
	The control flow graph shows this operation as a call to b.sort with reverse=True.
	Since b is an empty list, sorting it has no effect.
	b = []
4. Return (return a + b):
	The control flow graph indicates this return operation.
	a + b is the concatenation of lists a and b, resulting in [666].
	Therefore, the function returns [666].

After following the control flow of the function and given input parameters, the final output is: [666]
[/THOUGHT]
[ANSWER]
f([666]) == [666]
[/ANSWER]


100%|█████████▉| 249/250 [31:45<00:02,  2.38s/it]

[THOUGHT]
Let's execute code step by step:
1. Initial State:
	s = {'f': 1, 's': 1, 'a': 1}
	count = {}
2. First Loop Iteration (i = 'f'):
	i.islower() returns True.
	count['f'] = s.count('f') + count.get('f', 0).
	s.count('f') returns 1.
	count.get('f', 0) returns 0 because 'f' is not in count yet.
	count['f'] = 1 + 0 = 1.
	count = {'f': 1}
3. Second Loop Iteration (i = 's'):
	i.islower() returns True.
	count['s'] = s.count('s') + count.get('s', 0).
	s.count('s') returns 1.
	count.get('s', 0) returns 0 because 's' is not in count yet.
	count['s'] = 1 + 0 = 1.
	count = {'f': 1, 's': 1}
4. Third Loop Iteration (i = 'a'):
	i.islower() returns True.
	count['a'] = s.count('a') + count.get('a', 0).
	s.count('a') returns 1.
	count.get('a', 0) returns 0 because 'a' is not in count yet.
	count['a'] = 1 + 0 = 1.
	count = {'f': 1, 's': 1, 'a': 1}

After following the control flow of the function and given input parameters, the final output is: {'f': 1, 's': 1, 'a': 1}
[/THOUGHT]
[ANSWER]
f({'f': 

100%|██████████| 250/250 [31:49<00:00,  7.64s/it]

{'correct': 122, 'total': 250, 'error': 8}


In [13]:
print(env.result)

{'correct': 122, 'total': 250, 'error': 8}
